A python script for generating Superbowl Football (or basketball, any sport) game squares. It is suitable for any number of entries because it programatically scales / converts entries to a 10x10 grid and also allows for the empties after scaling to be assigned to a non-entrant.

Input: Accepts a csv file with a list of names and number of squares to assign to that person. No column headers should be used.

Output: An excel file with with the names assigned

In [71]:
import random
import datetime
import time
import pandas as pd

# Get input csv and output csv names
from collections import defaultdict, Counter

#initialize variables
input_file = 'input.csv' 
out_file = 'squares.xlsx'
EMPTY = "PAWS" # Unsold squares (eg Sold 75 squares, remaining 25 squares are assigned to "PAWS")
grid = None

# read input, entries, num_entries
df = pd.read_csv(input_file,header=None)

# convert squares sold to a 100 square format (eg If 25 squares sold, then squares sold = num_entries * 4)
if df[1].sum() <= 25 : df[1] *= 4
if df[1].sum() <= 50 : df[1] *= 2

# append row for unsold squares assigned to EMPTY
unsold_row = {0:EMPTY, 1:100-df[1].sum()}
df = df.append(unsold_row, ignore_index=True)

# convert df to list of tuples entries
entries = list(df.itertuples(index=False, name=None))

# A seed for the random generator. I fix the seed so rerunning the script will not
# allow for gaming the random generator to the benefit of the person running it.
SEED_TIME = datetime.datetime(2028, 2, 4, 16, 20, 0, 0)
entries = defaultdict(int)

# Generate seed for random based on date and time of drawing and
# number of people participating
t = SEED_TIME.timetuple()
seed = int(time.mktime(t) * df.shape[0])
rdm = random.Random(seed)

# generate a list of possible last digit scores for a game
border_boxes = [(nfc, afc) for nfc in range(0, 10) for afc in range(0, 10)]

# shuffle entries and empty_boxes like a deck of cards to drive random number assignments
rdm.shuffle(border_boxes)
rdm.shuffle(entries)
entries_boxes = defaultdict(list)

# sample num of boxes from empty_boxes
for s, n in entries :
    entries_boxes[s] += [border_boxes.pop(0) for _ in range(n)] 

# make empty 10x10 grid
grid = [['' for _ in range(0, 10)] for _ in range(0, 10)]

# assign entries to element in grid
for s, boxes in entries_boxes.items():
    for r, c in boxes:
        grid[r][c] = s
        
# output grid to excel spreadsheet
pd.DataFrame(data=grid).to_excel(out_file)
